# Amazon Bandit Experiment
Test contrastive vs reconstruction embeddings on real Amazon product data

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

from src.embeddings import get_extractor
from src.models import LinearContextualBandit, SimpleNeuralBandit
from src.datasets import AmazonDataset

In [2]:
print(f"MPS available: {torch.backends.mps.is_available()}")

MPS available: True


## 1. Load Amazon Data

In [3]:
! rm ../data/amazon/amazon_items.json

In [4]:
# Download real Amazon Electronics data (streams, stops at n_items_per_category)
# Default: 3 categories x 3333 items = ~10K total items
dataset = AmazonDataset(
    categories=['Electronics'],  # Start with one category
    n_items_per_category=3000,
    cache_dir='../data/amazon'
)
print(f"Loaded {len(dataset)} items")

# Get text for embeddings
texts = dataset.get_item_texts()
#print(f"Sample: {texts[0][:200]}...")

Electronics: 5282it [00:07, 687.61it/s]                          


Electronics: Downloaded 3000 items
Saved 3000 items to ../data/amazon/amazon_items.json
Loaded 3000 items


In [5]:
print(f"Sample: {texts[0][:200]}...")

Sample: FS-1051 FATSHARK TELEPORTER V3 HEADSET. Teleporter V3 The “Teleporter V3” kit sets a new level of value in the FPV world with Fat Shark renowned performance and quality. The fun of FPV is experienced ...


## 2. Compute Embeddings

In [6]:
# Contrastive (expected to work well)
simcse = get_extractor('simcse')
simcse_embs = simcse.encode(texts)
print(f"SimCSE embeddings: {simcse_embs.shape}")

# Reconstruction-based (expected to struggle)
bert = get_extractor('bert')
bert_embs = bert.encode(texts)
print(f"BERT embeddings: {bert_embs.shape}")

✓ SimCSE: Using MPS


SimCSE: 100%|██████████| 24/24 [07:17<00:00, 18.22s/it]


SimCSE embeddings: (3000, 768)
✓ BERT: Using MPS


BERT: 100%|██████████| 24/24 [02:51<00:00,  7.16s/it]

BERT embeddings: (3000, 768)


## 3. Compute Effective Dimension

In [7]:
from src.analysis.eigenvalues import compute_eigenvalue_spectrum, compute_effective_dimension

# Returns (eigenvalues, eigenvectors) - unpack the tuple
bert_eigs, _ = compute_eigenvalue_spectrum(bert_embs)
simcse_eigs, _ = compute_eigenvalue_spectrum(simcse_embs)

bert_deff = compute_effective_dimension(bert_eigs)
simcse_deff = compute_effective_dimension(simcse_eigs)

print(f"BERT d_eff: {bert_deff:.1f}")
print(f"SimCSE d_eff: {simcse_deff:.1f}")

BERT d_eff: 17.1
SimCSE d_eff: 43.3


## 4. Run Bandit Experiment

In [15]:
def run_bandit_experiment(embeddings, dataset, n_rounds=5000, n_candidates=500, seed=42):
    """
    Run Thompson Sampling bandit with given embeddings.
    
    Key design choices for valid regret measurement:
    - User preference is a random embedding direction (tests embedding geometry)
    - Reward depends on embedding similarity (directly tests hypothesis)
    - Regret computed using EXPECTED rewards (not stochastic samples)
    - Oracle best = max expected reward over candidates
    
    Args:
        embeddings: (N, d) array of item embeddings
        dataset: AmazonDataset instance (only used for len())
        n_rounds: Number of recommendation rounds
        n_candidates: Number of candidate items per round
        seed: Random seed for reproducibility
        
    Returns:
        Dict with 'rewards', 'regrets', 'cumulative_regret', 'final_regret'
    """
    np.random.seed(seed)
    
    # Initialize bandit
    bandit = LinearContextualBandit(embedding_dim=embeddings.shape[1], algorithm='ts')
    
    # Tracking
    rewards = []
    regrets = []
    cumulative_regret = 0
    
    for t in tqdm(range(n_rounds), desc="Bandit rounds"):
        # Sample random user preference (in embedding space)
        user_pref = np.random.randn(embeddings.shape[1])
        user_pref = user_pref / np.linalg.norm(user_pref)  # Normalize to unit vector
        
        # Get candidate items
        candidates = np.random.choice(len(dataset), n_candidates, replace=False)
        candidate_embs = embeddings[candidates]
        
        # Select action using bandit
        action_idx = bandit.select_arm(candidate_embs)
        selected_item = candidates[action_idx]
        
        # Compute EXPECTED rewards for all candidates (for regret)
        # Expected reward = sigmoid(similarity * scale)
        scale = 5.0  # Controls reward sensitivity
        similarities = candidate_embs @ user_pref  # (n_candidates,)
        expected_rewards = 1.0 / (1.0 + np.exp(-scale * similarities))  # Sigmoid
        
        # Oracle best action (for regret computation)
        best_expected = np.max(expected_rewards)
        selected_expected = expected_rewards[action_idx]
        
        # Instantaneous regret (using expected values)
        instant_regret = best_expected - selected_expected
        cumulative_regret += instant_regret
        
        # Sample actual reward (Bernoulli with prob = expected_reward)
        prob = expected_rewards[action_idx]
        reward = 1.0 if np.random.random() < prob else 0.0
        
        # Update bandit
        bandit.update(candidate_embs[action_idx], reward)
        
        # Track
        rewards.append(reward)
        regrets.append(instant_regret)
    
    return {
        'rewards': np.array(rewards),
        'regrets': np.array(regrets),
        'cumulative_regret': np.cumsum(regrets),
        'final_regret': cumulative_regret
    }

In [16]:
# Run experiments
print("Running BERT bandit...")
bert_rewards, bert_regrets = run_bandit_experiment(bert_embs, dataset)

print("Running SimCSE bandit...")
simcse_rewards, simcse_regrets = run_bandit_experiment(simcse_embs, dataset)

Bandit rounds:   0%|          | 0/5000 [00:00<?, ?it/s]

Running BERT bandit...


Bandit rounds: 100%|██████████| 5000/5000 [7:12:48<00:00,  5.19s/it]       


ValueError: too many values to unpack (expected 2)

## 5. Plot Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cumulative regret
ax1 = axes[0]
ax1.plot(np.cumsum(bert_regrets), label=f'BERT (d_eff={bert_deff:.0f})', color='red')
ax1.plot(np.cumsum(simcse_regrets), label=f'SimCSE (d_eff={simcse_deff:.0f})', color='blue')
ax1.set_xlabel('Round')
ax1.set_ylabel('Cumulative Regret')
ax1.set_title('Cumulative Regret: Contrastive vs Reconstruction')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rolling average reward
ax2 = axes[1]
window = 200
bert_rolling = np.convolve(bert_rewards, np.ones(window)/window, mode='valid')
simcse_rolling = np.convolve(simcse_rewards, np.ones(window)/window, mode='valid')
ax2.plot(bert_rolling, label='BERT', color='red', alpha=0.7)
ax2.plot(simcse_rolling, label='SimCSE', color='blue', alpha=0.7)
ax2.set_xlabel('Round')
ax2.set_ylabel('Average Reward (200-round window)')
ax2.set_title('Learning Curves')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/plots/amazon_bandit_results.png', dpi=150)
plt.show()

print(f"\nFinal cumulative regret:")
print(f"  BERT: {np.sum(bert_regrets):.1f}")
print(f"  SimCSE: {np.sum(simcse_regrets):.1f}")
print(f"  Reduction: {100*(np.sum(bert_regrets) - np.sum(simcse_regrets))/np.sum(bert_regrets):.1f}%")